In [1]:
#If you are using SageMath

Z11=Integers(11) #integers modulo 11

In [4]:
U11=Z11.unit_group() #group of invertible elements

In [5]:
U11

Multiplicative Abelian group isomorphic to C10

In [9]:
li=[x for x in U11]

In [11]:
type(li[2])

<class 'sage.groups.abelian_gps.values.AbelianGroupWithValues_class_with_category.element_class'>

In [14]:
a=Z11(5)

In [17]:
a.multiplicative_order()

5

In [19]:
(5**5)%11

1

In [20]:
#From Lagrange theorem we know that the orders of elements in U11 are only 1, 2, 5 or 10.

In [22]:
[k for k in range(1,11) if Z11(k).multiplicative_order() == 5]

[3, 4, 5, 9]

In [23]:
[k for k in range(1,11) if Z11(k).multiplicative_order() == 1]

[1]

In [24]:
[k for k in range(1,11) if Z11(k).multiplicative_order() == 10]

[2, 6, 7, 8]

In [25]:
[k for k in range(1,11) if Z11(k).multiplicative_order() == 2]

[10]

In $(\mathbb{Z}/11)^{\times}$ we have elements of orders 1,2,5 or 10:

1. Elements of order $1$: 1.
2. Elements of order $2$: 10.
3. Elements of order $5$: 3,4,5,9.
4. Elements of order $10$: 2,6,7,8.

In particular we have $4$ generators in this group.

In [26]:
def DiffieHellman(g,a):
    return g**a

In [49]:
[k for k in range(1,200) if (2**(Primes()[k])-1).is_prime()]

[1, 2, 3, 5, 6, 7, 10, 17, 23, 27, 30, 97, 110]

In [50]:
p=2**(Primes()[97])-1

In [62]:
p

6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151

In [63]:
(p-1).factor()

2 * 3 * 5^2 * 11 * 17 * 31 * 41 * 53 * 131 * 157 * 521 * 1613 * 2731 * 8191 * 42641 * 51481 * 61681 * 409891 * 858001 * 5746001 * 7623851 * 34110701 * 308761441 * 2400573761 * 65427463921 * 108140989558681 * 145295143558111 * 173308343918874810521923841

## Finding a generator of a multiplicative group.
For every prime $p$ the multiplicative group $\Phi(p)=(\mathbb{Z}/p)^{\times}$ is cyclic, hence there is an element $0<g<p$ such that $\Phi(p)=\{1,g,g^2,...g^{p-1}\}$.

Problem: how can we produce an element $g$?

Solution 1: brute force algorithm: we check with fast exponentiation which elements have order exactly $p-1$
To do that we need to check for each $a$ whether $a^k\equiv 1\textrm{ mod }p$ for each $k\mid p-1$ (we use essentially the fact that $\Phi(p)$ is a group.

Solution 2: if we know the factorization of the number $p-1$ we can run a probabilistic algorithm which will solve the task

$p-1=\prod_{i=1}^{r}q_i^{e_i}$

In [64]:
def FindGen(p):
    fact=(p-1).factor()
    gammali=[]
    for i in range(0,len(fact)):
        b=1
        while b==1:
            a=randint(1,p-1)
            b=power_mod(a,(p-1)//fact[i][0],p)
        gammali+=[power_mod(a,(p-1)//(fact[i][0]**fact[i][1]),p)]
    gamma=prod(gammali)%p
    return gamma
#analysis of the algorithm on page 328 (Shoup)
#estimated complexity about O(len(p)^4) (excluding time spent on the factorization!)

In [65]:
Zp=Integers(p) #integers modulo p

In [70]:
Zp(3).multiplicative_order()==p-1

True

In [76]:
# We want to have randomized generators!
g=Zp(FindGen(p))

In [78]:
#Alice
a=randint(2,p-1)
e1=g**a

In [80]:
#Bob
b=randint(2,p-1)
e2=g**b

In [82]:
#Alice
eA=e2**a

In [83]:
#Bob
eB=e1**b

In [84]:
eA==eB

True

In [85]:
#shared secret
e=eA #(=eB)

In [86]:
#Eve wants to find a number e from the knowledge of g,e1 and e2 (she knows Zp as well)

In [87]:
g

349562490585959426777829028036965234265261567416605082896816963518129585412075843706919538826200790579503432236143344887231414002183367994463647966489186423

In [88]:
e1

5629069575713336272317637809287278048068489755412570801618030884637950557863493592283244010011428452984866876957856704464977064356886001898427600684797015961

In [89]:
e2

5679846815634052100953939937378371619503648171807328183257248001693473194884830746695692694445721393017513688855651985898971338740326381269203141861027670135

In [90]:
p

6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151

In [91]:
#Eve needs to find the exponents a,b and compute g**(a*b) (this is e)

# Discrete logarithm for the group $(\mathbb{Z}/p)^{\times}$

Algorithms:
1. Brute force
2. Baby step, giant step

In [92]:
#we need to assume that alpha can be obtained from gamma (i.e. there is a number x s.t. gamma**x == alpha)
def BruteForceDLP(gamma,alpha): # we produce an element a such that gamma**a == alpha
    beta=1
    i=0
    while beta!=alpha: #the loop will never stop if the assumptions are not satisfied
        beta*=gamma
        i+=1
    return i

In [95]:
Z=Integers(19)
BruteForceDLP(Z(2),Z(2)**3)

3

In [96]:
Z=Integers(19)
BruteForceDLP(Z(2),Z(2)**197)

17

In [97]:
197%(19-1)

17

In [117]:
Z=Integers(Primes()[300000000])
%timeit BruteForceDLP(Z(1919),Z(1919)**1971111)

1 loop, best of 3: 4.31 s per loop


In [108]:
Z

Ring of integers modulo 4222234763

In [115]:
RealField(30)(log(p,10))

156.83663

In [116]:
RealField(30)(log(4222234763,10))

9.6255424

We can say that our DH scheme is safe for the DLP approach with the number p we've got (at least in the brute force attack)

## Baby step, giant step method

To solve the discrete logarithm problem one can apply the baby step-giant step method. 

Suppose we have a generator $g$ and a group $G$ spanned by $g$, which is of size $q$. We fix an integer $m$, roughly of the size $\sqrt{q}$ (hence $q/m$ is also of the size $\sqrt{q}$).

In the first part we build a collection of powers $g^i$ of $g$ where $0\leq i\leq m-1$. 
This can be done reasonably effectively using for example the dictionary.

In [125]:
#Shoup 11.2.2

#baby step - build a dictionary T
def DictT(g,q):
    m=floor(sqrt(q)*1.0)
    T=dict([])
    b=1
    for i in range(0,m-2):
        T[b]=i
        b*=g
    return T

Next, we perform the ''giant step'' part, where we look for the appropriate exponent, using the precomputed data structure $T$ of small powers of $g$.

In [126]:
def GiantStep(g,q,e):
    m=floor(sqrt(q)*1.0)
    g1=g**(-m)
    beta=e
    j=0
    T=DictT(g,q)
    if not T.has_key(beta):
        i=-1
    else:
        i=T[beta]
    while i==-1:
        beta*=g1
        j+=1
        if not T.has_key(beta):
            i=-1
        else:
            i=T[beta]
    x=j*m+i #this computing the exponent x such that g**x == e
    return x

In [127]:
Z=Integers(Primes()[300000000])
%timeit BruteForceDLP(Z(1919),Z(1919)**1971111)

1 loop, best of 3: 3.85 s per loop


In [129]:
%timeit GiantStep(Z(1919),Primes()[300000000]-1,Z(1919)**1971111)

1 loop, best of 3: 197 ms per loop


In [130]:
4/0.1

40.0000000000000

In [131]:
[k for k in range(1,200) if (2**(Primes()[k])-1).is_prime()]

[1, 2, 3, 5, 6, 7, 10, 17, 23, 27, 30, 97, 110]

In [ ]:
p1=2**(Primes()[10])-1
Z=Integers(p1)
g=32749237492
a=197111178433
GiantStep(Z(g),p1-1,Z(g)**a)

With the  prime p with $600$ digits we are pretty safe in terms of DLP attack even with baby step, giant step method.